<a href="https://colab.research.google.com/github/etomoscow/DL-in-NLP/blob/master/hw3/task4_sentiment_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment 2.4: Text classification via CNN (20 points)

In this assignment you should perform sentiment analysis of the IMDB reviews based on CNN architecture. Read carefully [Convolutional Neural Networks for Sentence Classification](https://arxiv.org/pdf/1408.5882.pdf) by Yoon Kim.

In [0]:
import numpy as np
import torch
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score 

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext import datasets
from torchtext.data import Field, LabelField
from torchtext.data import BucketIterator

### Preparing Data

In [0]:
TEXT = Field(sequential=True,
             lower=True,
             batch_first=True,
             fix_length=200,
             include_lengths=True
             )
LABEL = LabelField(batch_first=True)

In [3]:
train, tst = datasets.IMDB.splits(TEXT, LABEL)
trn, vld = train.split()

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:08<00:00, 10.3MB/s]


In [4]:
%%time
TEXT.build_vocab(trn)

CPU times: user 1.14 s, sys: 30.9 ms, total: 1.17 s
Wall time: 1.18 s


In [0]:
LABEL.build_vocab(trn)

### Creating the Iterator (2 points)

Define an iterator here

In [0]:
train_iter, val_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(64, 64, 64),
        sort=False,
        sort_key= lambda x: len(x.text),
        sort_within_batch=False,
        device='cuda',
        repeat=False
)

In [0]:
batch = next(train_iter.__iter__())

### Define CNN-based text classification model (8 points)

In [0]:
class CNN(nn.Module):
    def __init__(self, V, D, kernel_sizes, stride=1, padding=0, dropout=0.5, out_channels=1):
        super(CNN, self).__init__()
        
        self.V = V
        self.D = D
        self.kernel_sizes = kernel_sizes
        self.stride = stride
        self.padding = padding
        self.dropout = dropout
        self.out_channels = out_channels
    
        self.embs = nn.Embedding(V, D, padding_idx=1)
        self.conv1 = nn.Conv2d(1, out_channels,(kernel_sizes[0], D))
        self.conv2 = nn.Conv2d(1, out_channels, (kernel_sizes[1], D))
        self.conv3 = nn.Conv2d(1, out_channels, (kernel_sizes[2], D))
        self.dp = nn.Dropout(dropout)
        self.lin = nn.Linear(len(kernel_sizes)*out_channels, 2)
    
    
    def convolution_block(self, input, conv):
        conved = conv(input)
        act = F.relu(conved.squeeze(3))
        out = F.max_pool1d(act, act.size()[2]).squeeze(2)

        return out
    def forward(self, x):
        
        x = self.embs(x).unsqueeze(1)
        out1 = self.convolution_block(x, self.conv1)
        out2 = self.convolution_block(x, self.conv2)
        out3 = self.convolution_block(x, self.conv3)

        concat = self.dp(torch.cat((out1, out2, out3), 1))
        logit = self.lin(concat)
        return logit

In [0]:
kernel_sizes = [3,4,5]
vocab_size = len(TEXT.vocab)
dropout = 0.5
dim = 300

model = CNN(vocab_size, dim, kernel_sizes, dropout)

In [19]:
model.cuda()

CNN(
  (embs): Embedding(201765, 300, padding_idx=1)
  (conv1): Conv2d(1, 1, kernel_size=(3, 300), stride=(1, 1))
  (conv2): Conv2d(1, 1, kernel_size=(4, 300), stride=(1, 1))
  (conv3): Conv2d(1, 1, kernel_size=(5, 300), stride=(1, 1))
  (dp): Dropout(p=0.5, inplace=False)
  (lin): Linear(in_features=3, out_features=2, bias=True)
)

### The training loop (3 points)

Define the optimization function and the loss functions.

In [0]:
opt = optim.Adam(model.parameters(), lr=0.05)
loss_func = nn.CrossEntropyLoss()

Think carefully about the stopping criteria. 

In [0]:
epochs = 10

In [0]:
def train_model(model, optimizer, loss_func, num_epochs, trainset, valset):
    %%time
    train_loss, valid_loss = [], []
    for epoch in range(1, num_epochs + 1):
        train_acc, val_acc = [], []
        running_loss = 0.0
        running_corrects = 0
        model.train() 
        for batch in trainset:         
        
            x = batch.text[0]
            y = batch.label
        
            opt.zero_grad()
            preds = model(x)
            loss = loss_func(preds, y)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
            n_correct = (torch.max(preds, 1)[1].view(y.size()).data == y.data).float().sum().detach().cpu().numpy()
            train_acc.append(100.0 * n_correct/len(batch))

        epoch_loss = running_loss / len(trn)
        train_loss.append(epoch_loss)

        val_loss = 0.0
        model.eval()
        correct = 0
        total = 0 
        for batch in val_iter:
            with torch.no_grad():
                x = batch.text[0]
                y = batch.label
        
                preds = model(x)
                loss = loss_func(preds, y)
                val_loss += loss.item()

                n_correct = (torch.max(preds, 1)[1].view(y.size()).data == y.data).float().sum().detach().cpu().numpy()
                val_acc.append(100.0 * n_correct/len(batch))

        val_loss /= len(vld)
        valid_loss.append(val_loss)
        print('Epoch: {}, Training Loss: {}, Training Accuracy: {}, Validation Loss: {}, Validation Accuracy: {}'.format(epoch, round(epoch_loss, 5), round(np.mean(train_acc), 2), round(val_loss, 5), round(np.mean(val_acc), 2)))

In [22]:
train_model(model, opt, loss_func, epochs, train_iter, val_iter)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs
Epoch: 1, Training Loss: 0.01247, Training Accuracy: 49.77, Validation Loss: 0.01126, Validation Accuracy: 49.83
Epoch: 2, Training Loss: 0.01205, Training Accuracy: 50.75, Validation Loss: 0.01096, Validation Accuracy: 48.63
Epoch: 3, Training Loss: 0.01219, Training Accuracy: 52.93, Validation Loss: 0.01172, Validation Accuracy: 53.03
Epoch: 4, Training Loss: 0.01445, Training Accuracy: 53.81, Validation Loss: 0.01283, Validation Accuracy: 51.22
Epoch: 5, Training Loss: 0.01604, Training Accuracy: 54.01, Validation Loss: 0.01227, Validation Accuracy: 50.45
Epoch: 6, Training Loss: 0.01544, Training Accuracy: 53.99, Validation Loss: 0.01374, Validation Accuracy: 51.64
Epoch: 7, Training Loss: 0.01543, Training Accuracy: 54.58, Validation Loss: 0.01923, Validation Accuracy: 49.83
Epoch: 8, Training Loss: 0.01565, Training Accuracy: 55.7, Validation Loss: 0.01253, Validation Accuracy: 52.28
Epoch: 9, Training Loss: 0.0172, 

### Calculate performance of the trained model (2 points)

In [0]:
def evaluate_model(model, testset):
    model.eval()
    accuracies, f1_scores, precisions, recalls = [], [], [], []
    for batch in testset:
        with torch.no_grad():
            x = batch.text[0]
            y = batch.label
            preds = model(x)
            n_correct = (torch.max(preds, 1)[1].view(y.size()).data == y.data).float().sum().detach().cpu().numpy()
            accuracies.append(100.0 * n_correct/len(batch))
            logit = torch.max(preds, 1)[1].view(y.size()).data.detach().cpu().numpy()
            target = y.data.detach().cpu().numpy()
            f1_scores.append(f1_score(logit, target, zero_division=1))
            precisions.append(precision_score(logit, target, zero_division=0))
            recalls.append(recall_score(logit, target, zero_division=1))

    print('Average classification scores:\nAccuracy:{}, \nPrecision:{},\nRecall:{},\nF1-score:{}'.format(round(np.mean(accuracies),3),
                                                                                                     round(100.0 *np.mean(f1_scores),3),
                                                                                                     round(100.0 *np.mean(precisions),3),
                                                                                                     round(100.0 *np.mean(recalls),3)))

In [0]:
evaluate_model(model, test_iter)

Write down the calculated performance

### Accuracy: 51.219
### Precision: 16.324
### Recall: 9.858
### F1: 49.957

### Experiments (5 points)

Experiment with the model and achieve better results. Implement and describe your experiments in details, mention what was helpful.

### 1. Bad results on the baseline. Let's tune the model. Actions: lower the embedding dimension, lower the learning rate, increase the number of epochs.Results: Accuracy:70.827, Precision:42.507, Recall:37.075, F1-score:50.031

In [0]:
kernel_sizes = [3,4,5]
vocab_size = len(TEXT.vocab)
dropout = 0.5
dim = 256

model = CNN(vocab_size, dim, kernel_sizes, dropout)
model.cuda()
opt = optim.Adam(model.parameters(), lr=0.0002)
loss_func = nn.CrossEntropyLoss()

In [34]:
train_model(model, opt, loss_func, 30, train_iter, val_iter)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.87 µs
Epoch: 1, Training Loss: 0.01175, Training Accuracy: 50.48, Validation Loss: 0.01079, Validation Accuracy: 53.44
Epoch: 2, Training Loss: 0.01079, Training Accuracy: 53.86, Validation Loss: 0.01041, Validation Accuracy: 62.01
Epoch: 3, Training Loss: 0.01056, Training Accuracy: 55.34, Validation Loss: 0.01028, Validation Accuracy: 63.45
Epoch: 4, Training Loss: 0.01042, Training Accuracy: 57.42, Validation Loss: 0.01017, Validation Accuracy: 64.83
Epoch: 5, Training Loss: 0.01037, Training Accuracy: 58.3, Validation Loss: 0.01012, Validation Accuracy: 65.34
Epoch: 6, Training Loss: 0.01023, Training Accuracy: 60.33, Validation Loss: 0.01005, Validation Accuracy: 65.49
Epoch: 7, Training Loss: 0.01019, Training Accuracy: 60.65, Validation Loss: 0.00996, Validation Accuracy: 66.46
Epoch: 8, Training Loss: 0.01008, Training Accuracy: 61.68, Validation Loss: 0.00987, Validation Accuracy: 66.96
Epoch: 9, Training Loss: 0.00

In [35]:
evaluate_model(model, test_iter)

Average classification scores:
Accuracy:70.827, 
Precision:42.507,
Recall:37.075,
F1-score:50.031
